In [11]:
!python -m venv myenv

In [12]:
!pip install duckdb pandas polars numpy pyarrow

In [13]:
!aws s3 ls --profile ccprofile


The config profile (ccprofile) could not be found


In [3]:
import duckdb
import pandas as pd

In [5]:
con = duckdb.connect()
con.install_extension("aws")
con.load_extension("aws")

con.install_extension("s3")
con.load_extension("s3")

con.install_extension("httpfs")
con.load_extension("httpfs")

In [6]:
CRAWL_SET = "CC-MAIN-2024-42"
CC_URL_S3_BASE="s3://"
CC_URL_HTTP_BASE="https://data.commoncrawl.org/"
CC_INDEX_URL = f"{CC_URL_HTTP_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_BASE_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/"
CC_INDEX_URL

'https://data.commoncrawl.org/commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [7]:
CC_INDEX_S3_URL

's3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [8]:
print(f"{CC_INDEX_S3_BASE_URL}")

s3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/


In [9]:
# Configure S3 settings for DuckDB
con.execute("""
    CALL load_aws_credentials('ccprofile');
""")

In [10]:
con.execute("""
  DROP SECRET secret3;
  CREATE SECRET secret3 (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN,
    CHAIN 'sso'
  );
            """)

InvalidInputException: Invalid Input Error: Failed to remove non-existent secret with name 'secret3'

In [68]:
# Read first few rows of Common Crawl index without downloading full dataset
try:
    query = f"""
        SELECT *
        FROM read_parquet('{CC_INDEX_S3_URL}')
        LIMIT 10
    """
    df = con.execute(query).fetchdf()
    print("First 5 rows of Common Crawl index:")
    print(df.head())
except Exception as e:
    print(f"Error reading Common Crawl index: {str(e)}")


First 5 rows of Common Crawl index:
                                         url_surtkey  \
0  com,blogspot,blogdocappacete)/2012/11/a-breve-...   
1  com,blogspot,blogdocappacete)/2012/11/burguesi...   
2  com,blogspot,blogdocappacete)/2012/11/deputado...   
3  com,blogspot,blogdocappacete)/2012/11/enquanto...   
4  com,blogspot,blogdocappacete)/2012/11/matanca-...   

                                                 url  \
0  https://blogdocappacete.blogspot.com/2012/11/a...   
1  https://blogdocappacete.blogspot.com/2012/11/b...   
2  https://blogdocappacete.blogspot.com/2012/11/d...   
3  https://blogdocappacete.blogspot.com/2012/11/e...   
4  https://blogdocappacete.blogspot.com/2012/11/m...   

                  url_host_name url_host_tld url_host_2nd_last_part  \
0  blogdocappacete.blogspot.com          com               blogspot   
1  blogdocappacete.blogspot.com          com               blogspot   
2  blogdocappacete.blogspot.com          com               blogspot   
3  blo

In [75]:
con.execute(f"""
  CREATE TABLE IF NOT EXISTS t1 as 
  SELECT *
    FROM read_parquet('{CC_INDEX_S3_URL}')
    LIMIT 100;
  DESCRIBE t1;
""").df()

,column_name,column_type,null,key,default,extra
0,url_surtkey,VARCHAR,YES,None,None,None
1,url,VARCHAR,YES,None,None,None
2,url_host_name,VARCHAR,YES,None,None,None
3,url_host_tld,VARCHAR,YES,None,None,None
4,url_host_2nd_last_part,VARCHAR,YES,None,None,None
5,url_host_3rd_last_part,VARCHAR,YES,None,None,None
6,url_host_4th_last_part,VARCHAR,YES,None,None,None
7,url_host_5th_last_part,VARCHAR,YES,None,None,None
8,url_host_registry_suffix,VARCHAR,YES,None,None,None
9,url_host_registered_domain,VARCHAR,YES,None,None,None


In [9]:
#!cut -d' ' -f3- ../cc/cdx-00000 > ../cc/cdx-00000.jsonl 

# IGNORE EVERYTHING BELOW THIS LINE

In [ ]:
!duckdb -c "\
  create table meta as SELECT * FROM '../cc/cdx-00000.jsonl'; \
  create view status_vw as Select status, count(status) as cnt from meta group by status order by cnt desc; \
  select * from meta; \
  select * from status_vw; \
" {DB_FILE}

In [ ]:
!duckdb -c ".databases" {DB_FILE}
#!duckdb -c ".open hello.db"

In [ ]:
!duckdb -c " \
create table metastats as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta group by tstatus order by cstatus desc; \
select * from metastats; \
" {DB_FILE}

In [ ]:
!duckdb -c " \
create table metastats_norobotstxt as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta where url not like '%robots.txt' group by tstatus order by cstatus desc; \
select * from metastats_norobotstxt; \
" {DB_FILE}

In [ ]:
#!duckdb -c "drop table metastats" {DB_FILE}

In [ ]:
first_line = ""
with open("../cc/cdx-00000", "r") as file:
    first_line = file.readline()

print(first_line)

In [ ]:
!echo "{first_line}" | cut -d' ' -f3- 

In [ ]:
import json
s = '{"url": "http://1.0.0.0/robots.txt", "mime": "text/html", "mime-detected": "text/html", "status": "403", "digest": "UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ", "length": "2706", "offset": "890", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz"}'
json.loads(s)